In [1]:
import drawSvg
from drawSvg.widgets import DrawingWidget
import sys
sys.path.insert(0,"..")
from hyperbolic.poincare.shapes import *
from constructions import *
from Triangle import *
from deltaNeigbourhood import *


backcolor = drawSvg.RadialGradient(0,0,1)
backcolor.addStop(0, '#ff9832', opacity=1)
backcolor.addStop(1, '#ff7f00', opacity=1)

polycolor1 = drawSvg.RadialGradient(0,0,1)
polycolor1.addStop(0, '#2c6493', opacity=1)
polycolor1.addStop(1, '#377eb8', opacity=1)

polycolor2 = drawSvg.RadialGradient(0,0,1)
polycolor2.addStop(0, '#e41a1c', opacity=0.75)
polycolor2.addStop(1, '#e94749', opacity=0.5)

backgroundStyle = dict(stroke='#cc6500', stroke_width=0.01, fill=backcolor)
polyStyle1 = dict(stroke='black', stroke_opacity=0.5, stroke_width=0.005,  fill=polycolor1)
polyStyle2 = dict(stroke='black', stroke_opacity=0.5, stroke_width=0.005, fill=polycolor2)
lineStyle1 = dict(hwidth=0.04, fill='#984ea3')
lineStyle2 = dict(hwidth=0.04, fill='yellow')
pointStyle1 = dict(hradius=0.02, fill='#984ea3')
pointStyle2 = dict(hradius=0.02, fill='yellow')

delta=0.3


# Create drawing
d = drawSvg.Drawing(2.1, 2.5, origin='center')
d.setRenderSize(500)
d.append(drawSvg.Circle(0, 0, 1, **backgroundStyle))
g=drawSvg.Group()
d.append(g)

#setup for widegt
PList=[]
clicks=0
approximated=False
text='click inside the circle to add a point'
deltatext='delta is set to = {}'.format(delta)

def drawText(text1, text2):
    g.draw(drawSvg.Text(text1, 0.1, 0, 1.1, center=0.7))
    g.draw(drawSvg.Text(text2, 0.1, 0, 1.2, center=0.7))
    

def drawLines(points):
    for i,p1 in enumerate(points):
        for p2 in points[i+1:]:
            g.draw(Line.fromPoints(*p1, *p2, segment=True), **lineStyle1)
    for p1 in points:
        g.draw(p1, **pointStyle2)

def drawTriAndNbh(points, edgeNum):
    global approxdelta
    Tri=Triangle.fromVertices(points)
    if clicks==3:
        approxdelta = Tri.approx()
    if approximated:
        offset = approxdelta
    else:
        offset = delta
    g.draw(Tri,**polyStyle1)
    g.draw(deltaNbh(Tri, offset, edgeNum), **polyStyle2)


        
widget = DrawingWidget(d)
@widget.mousedown

def mousedown(widget, x, y, info):
    global clicks, approximated, text, deltatext
    if (x**2 + y**2) ** 0.5 < 1:
        clicks+=1
        if clicks<3:
            PList.append(Point(x, y))
            g.children.clear()
            drawLines(PList)
            drawText(text, deltatext)
        elif clicks==3:
            PList.append(Point(x, y))
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)
        else:
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)       
    else:
        approximated = not approximated
        if approximated:
            text='click to change to set delta'
            deltatext='approximated delta = {}'.format(approxdelta)
        else:
            text='click to change to approximated delta'
            deltatext='set delta = {}'.format(delta)
        if len(PList)==3:
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)
        else:
            g.children.clear()
            drawLines(PList)
            drawText(text, deltatext)

    widget.refresh()
        
        
@widget.mousemove
def mousemove(widget, x, y, info):
    global text, deltatext
    if (x**2 + y**2) ** 0.5  < 1:
        if len(PList)<3:
            text='click to add point'
            g.children.clear()
            g.draw(Point(x, y),**pointStyle2)
            drawLines([Point(x, y)] + PList)
            drawText(text, deltatext)
        else:
            text='click to change edge of deltaneigbourhood'
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)    
    else:
        if len(PList)==3:
            if approximated:
                text='click to change to set delta'
                deltatext='approximated delta = {}'.format(approxdelta)
            else:
                text='click to change to approximated delta'
                deltatext='set delta = {}'.format(delta)
            g.children.clear()
            drawTriAndNbh(PList, clicks)
            drawText(text, deltatext)
        else:
            text='click inside the circle to add a point'
            g.children.clear()
            drawLines(PList)
            drawText(text, deltatext)

    widget.refresh()

widget


C:\ProgramData\Anaconda3\lib\site-packages\drawSvg\raster.py:18: RuntimeWarning: Failed to import CairoSVG. drawSvg will be unable to output PNG or other raster image formats. See https://github.com/cduck/drawSvg#prerequisites for more details.
Original OSError: no library called "cairo-2" was found
no library called "cairo" was found
no library called "libcairo-2" was found
cannot load library 'libcairo.so.2': error 0x7e
cannot load library 'libcairo.2.dylib': error 0x7e
cannot load library 'libcairo-2.dll': error 0x7e
  warnings.warn(msg, RuntimeWarning)


<IPython.core.display.Javascript object>

DrawingWidget()